# AgavePy

<img style="display:inline;vertical-align:center" src="python-logo-generic.svg" width="150px"> 
<span style="font-size:28pt">+</span>
<img style="display:inline;vertical-align:bottom" src="logo-white-lg.png" width="160px">

These are instructions and a basic example of how to run `agavepy`.

## Setting up

Fill the following variables to the values for your system. The default ones work against a sandbox.

In [1]:
USERNAME = 'jdoe'
PASSWORD = 'abcde'

This is the IP of a machine with `ssh` access, to work as an storage system:

In [2]:
STORAGE_IP = '191.236.150.54'
STORAGE_USERNAME = 'jstubbs'

# home directory in STORAGE_IP
HOME_DIR = '/home/jstubbs'        

# credentials to access STORAGE_IP
STORAGE_PASSWORD = 'your password here'

Here are the parameters needed for an execution system.  In this example we point it to the same IP as the storage system.

In [3]:
EXECUTION_IP = STORAGE_IP
EXECUTION_USERNAME = STORAGE_USERNAME
EXECUTION_PASSWORD = STORAGE_PASSWORD

## Creating the agave object

Import the `agavepy` module:

In [4]:
import agavepy.agave as a
import json

Instantiate with your username and password.

`api_server` points to an agave server. In the current container, `dev.agave.tacc.utexas.edu` is defined in `/etc/hosts` pointing to an Agave sandbox.

In [5]:
agave = a.Agave(api_server='https://dev.agave.tacc.utexas.edu', 
                username=USERNAME,
                password=PASSWORD,
                verify=False)

Usually there are already clients defined. If you created them with `agavepy`, you already have the credentials to use those clients in a file `~/.agavepy`, which is used automatically.  Otherwise, you can create new ones.

List them with:

In [6]:
agave.clients.list()

[{u'_links': {u'self': {u'href': u'http://localhost:8000/clients/v2/DefaultApplication'},
   u'subscriber': {u'href': u'http://localhost:8000/profiles/v2/jdoe'},
   u'subscriptions': {u'href': u'http://localhost:8000/clients/v2/DefaultApplication/subscriptions/'}},
  u'callbackUrl': None,
  u'consumerKey': u'P9ngOdGkFVIBoRvp2AuZySeuNi0a',
  u'description': None,
  u'name': u'DefaultApplication',
  u'tier': u'Unlimited'},
 {u'_links': {u'self': {u'href': u'http://localhost:8000/clients/v2/eod'},
   u'subscriber': {u'href': u'http://localhost:8000/profiles/v2/jdoe'},
   u'subscriptions': {u'href': u'http://localhost:8000/clients/v2/eod/subscriptions/'}},
  u'callbackUrl': u'http://localhost:8000/oauth2/token',
  u'consumerKey': u'Oa2NWUf1zyRkLHyT1fQEEBpM4DUa',
  u'description': u'',
  u'name': u'eod',
  u'tier': u'Unlimited'},
 {u'_links': {u'self': {u'href': u'http://localhost:8000/clients/v2/myclient'},
   u'subscriber': {u'href': u'http://localhost:8000/profiles/v2/jdoe'},
   u'subscr

You can create a new client, in which case `agavepy` gets automatically connected to it:

In [7]:
agave.clients.delete(clientName='myclient')  # delete old client first (if any)
agave.clients.create(body={'clientName': 'myclient'})

{u'_links': {u'self': {u'href': u'http://localhost:8000/clients/v2/myclient'},
  u'subscriber': {u'href': u'http://localhost:8000/profiles/v2/jdoe'},
  u'subscriptions': {u'href': u'http://localhost:8000/clients/v2/myclient/subscriptions/'}},
 u'callbackUrl': u'',
 u'consumerKey': u'yYVhe6icR3my8wNA7Hz_Afddki0a',
 u'consumerSecret': u'KZI0G2MZFYVUnA7trGO5W1bkC6oa',
 u'description': u'',
 u'name': u'myclient',
 u'tier': u'Unlimited'}

The new client should appear now in the list. Let's check it:

In [8]:
[client for client in agave.clients.list() if client['name'] == 'myclient']

[{u'_links': {u'self': {u'href': u'http://localhost:8000/clients/v2/myclient'},
   u'subscriber': {u'href': u'http://localhost:8000/profiles/v2/jdoe'},
   u'subscriptions': {u'href': u'http://localhost:8000/clients/v2/myclient/subscriptions/'}},
  u'callbackUrl': u'',
  u'consumerKey': u'yYVhe6icR3my8wNA7Hz_Afddki0a',
  u'description': u'',
  u'name': u'myclient',
  u'tier': u'Unlimited'}]

## Using the API

### Systems

At this point, the object `agave` is connected to the API using a client.  As a test, let's list the systems associated to this client:

In [9]:
agave.systems.list()

[{u'_links': {u'self': {u'href': u'https://docker.example.com/systems/2.0/jfs.pysdk.endofday.local.compute.com'}},
  u'default': False,
  u'description': u'Execution system for local endofday runs',
  u'id': u'jfs.pysdk.endofday.local.compute.com',
  u'name': u'endofday local execution system',
  u'public': False,
  u'status': u'UP',
  u'type': u'EXECUTION'},
 {u'_links': {u'self': {u'href': u'https://docker.example.com/systems/2.0/endofday.local.storage.com'}},
  u'default': False,
  u'description': u'Storage system for local endofday runs',
  u'id': u'endofday.local.storage.com',
  u'name': u'endofday local storage system',
  u'public': False,
  u'status': u'UP',
  u'type': u'STORAGE'},
 {u'_links': {u'self': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system'}},
  u'default': False,
  u'description': u'Execution system for agavepy test',
  u'id': u'my.execution.system',
  u'name': u'Agavepy execution test system',
  u'public': False,
  u'status': u'UP',
  u'type'

This is the JSON object necessary to register a new storage system:

In [10]:
storage = {
   "id": "my.test.system",
   "name": "Agavepy test storage system",
   "status": "UP",
   "type": "STORAGE",
   "description": "Storage system for agavepy test",
   "site": "agaveapi.co",
   "storage":{
      "host": STORAGE_IP,
      "port": 22,
      "protocol": "SFTP",
      "rootDir": "/",
      "homeDir": HOME_DIR,
      "auth":{
         "username": STORAGE_USERNAME,
         "type": "PASSWORD",
         "password": STORAGE_PASSWORD
      }
   }
}

The actuall call to the Agave API is:

In [12]:
agave.systems.add(body=storage)

{u'_links': {u'credentials': {u'href': u'https://docker.example.com/systems/2.0/my.test.system/credentials'},
  u'metadata': {u'href': u'https://docker.example.com/meta/2.0/data/?q={"associationIds":"0001431720189499-242ac11c-0001-006"}'},
  u'owner': {u'href': u'https://docker.example.com/profiles/2.0/jdoe'},
  u'roles': {u'href': u'https://docker.example.com/systems/2.0/my.test.system/roles'},
  u'self': {u'href': u'https://docker.example.com/systems/2.0/my.test.system'}},
 u'description': u'Storage system for agavepy test',
 u'id': u'my.test.system',
 u'lastModified': datetime.datetime(2015, 5, 18, 12, 14, 9, 728000, tzinfo=tzoffset(None, -18000)),
 u'name': u'Agavepy test storage system',
 u'public': False,
 u'revision': 7,
 u'site': u'agaveapi.co',
 u'status': u'UP',
 u'storage': {u'auth': {u'type': u'PASSWORD'},
  u'homeDir': u'/home/jstubbs',
  u'host': u'191.236.150.54',
  u'mirror': False,
  u'port': 22,
  u'protocol': u'SFTP',
  u'proxy': None,
  u'publicAppsDir': None,
  u'r

Let's check that the storage system is up and that we have access by listing files:

In [13]:
[f.name for f in agave.files.list(systemId='my.test.system', filePath=HOME_DIR)]

[u'.',
 u'.bash_history',
 u'.bash_logout',
 u'.bash_profile',
 u'.bashrc',
 u'.pki',
 u'.viminfo',
 u'docker-latest',
 u'endofday',
 u'hello.txt',
 u'jdoe',
 u'pysdk_testsuiteapp',
 u'wc.py',
 u'wc.sh']

And let's be bold and add a file.

We create the file locally first:

In [14]:
import tempfile
temp = tempfile.mktemp()

with open(temp, 'w') as f:
    f.write('Hello world!')

and we upload it with:

In [15]:
response = agave.files.importData(
    systemId='my.test.system',
    filePath=HOME_DIR,
    fileName='hello.txt',
    fileToUpload=open(temp))
response

{u'_links': {u'history': {u'href': u'https://docker.example.com/files/2.0/history/system/my.test.system/hello.txt'},
  u'self': {u'href': u'https://docker.example.com/files/2.0/media/system/my.test.system/hello.txt'},
  u'system': {u'href': u'https://docker.example.com/systems/2.0/my.test.system'}},
 u'internalUsername': None,
 u'lastModified': datetime.datetime(2015, 5, 18, 12, 14, 56, 831000, tzinfo=tzoffset(None, -18000)),
 u'name': u'hello.txt',
 u'nativeFormat': u'raw',
 u'owner': u'jdoe',
 u'path': u'/home/jstubbs/hello.txt',
 u'source': u'http://191.236.147.52/tmpQrnS8h',
 u'status': u'STAGING_QUEUED',
 u'systemId': u'my.test.system',
 u'uuid': u'0001431720700707-242ac1113-0001-002'}

Note the field `response['uuid']`, which is an identifier that can be used to refer to the file.

Let's try downloading the file:

In [16]:
agave.files.download(systemId='my.test.system', filePath='hello.txt').text

u'Hello world!'

Now we register an execution system.  This is the JSON object:

In [17]:
execution = {
    "id": "my.execution.system",
    "name": "Agavepy execution test system",
    "status": "UP",
    "type": "EXECUTION",
    "description": "Execution system for agavepy test",
    "site": "agaveapi.co",
    "executionType": "CLI",
    "scratchDir": HOME_DIR,
    "workDir": HOME_DIR,
    "queues": [
        {
            "name": "debug",
            "maxJobs": 100,
            "maxUserJobs": 10,
            "maxNodes": 128,
            "maxMemoryPerNode": "2GB",
            "maxProcessorsPerNode": 128,
            "maxRequestedTime": "24:00:00",
            "customDirectives": "",
            "default": True
        }
    ],
    "login":{
        "host": EXECUTION_IP,
        "port": 22,
        "protocol": "SSH",
        "scratchDir": HOME_DIR,
        "workDir": HOME_DIR,
        "auth":{
            "username": EXECUTION_USERNAME,
            "type":"PASSWORD",
            "password": EXECUTION_PASSWORD
        }
    },
    "storage":{
        "host": STORAGE_IP,
        "port": 22,
        "protocol": "SFTP",
        "rootDir": "/",
        "homeDir": HOME_DIR,
        "auth":{
            "username": STORAGE_USERNAME,
            "type":"PASSWORD",
            "password": STORAGE_PASSWORD
        }
    },
    "maxSystemJobs": 100,
    "maxSystemJobsPerUser": 10,
    "scheduler": "FORK",
    "environment": "",
    "startupScript": "./bashrc"
}


The execution system is registered in the same way as the storage:

In [18]:
agave.systems.add(body=execution)

{u'_links': {u'credentials': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system/credentials'},
  u'metadata': {u'href': u'https://docker.example.com/meta/2.0/data/?q={"associationIds":"0001431721052429-242ac11c-0001-006"}'},
  u'owner': {u'href': u'https://docker.example.com/profiles/2.0/jdoe'},
  u'roles': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system/roles'},
  u'self': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system'}},
 u'description': u'Execution system for agavepy test',
 u'environment': None,
 u'executionType': u'CLI',
 u'id': u'my.execution.system',
 u'lastModified': datetime.datetime(2015, 5, 18, 12, 15, 10, 527000, tzinfo=tzoffset(None, -18000)),
 u'login': {u'auth': {u'type': u'PASSWORD'},
  u'host': u'191.236.150.54',
  u'port': 22,
  u'protocol': u'SSH',
  u'proxy': None},
 u'maxSystemJobs': 100,
 u'maxSystemJobsPerUser': 10,
 u'name': u'Agavepy execution test system',
 u'public': False,
 u'queues': [{u'cu

### Apps

Let's create a simple app that counts words.  The Python functions is:

In [19]:
%%writefile wc.py
import sys

def word_count(filename):
    return len(open(filename).read().split())  

if __name__ == '__main__':
    with open('out.txt', 'w') as out:
        out.write(str(word_count(sys.argv[1])))

Overwriting wc.py


We test it by creating a temporary file and passing its name:

In [20]:
temp = tempfile.mktemp()
with open(temp, 'w') as f:
    f.write('Lorem ipsum dolor sit amet, consectetur adipiscing elit.')

import wc
print(wc.word_count(temp))

8


Let's wrap the Python function in a shell script:

In [21]:
%%writefile wc.sh
python2 wc.py ${temp}

Overwriting wc.sh


And let's test it:

In [22]:
!chmod +x wc.sh      # make shell script executable
!temp=$temp ./wc.sh  # call wc.sh with env variable 'temp'
!cat out.txt         # display output created by wc.sh

8

Here we upload the files `wc.py` and `wc.sh` to the storage system:

In [24]:
for filename in ['wc.py', 'wc.sh']:
    agave.files.importData(
        systemId='my.test.system',
        filePath=HOME_DIR,
        fileName=filename,
        fileToUpload=open(filename))

Now we register an app with name `my.wc` using the previously registered storage and execution systems.  The app takes an input with id `temp`, matching the name of the input used in `wc.sh` above.

The JSON object is:

In [25]:
app = {
    "defaultNodes": 1, 
    "defaultMemoryPerNode": "2GB", 
    "datePublished": "", 
    "defaultQueue": "debug", 
    "author": "agavepy", 
    "shortDescription": "Word count app", 
    "label": "word-count", 
    "defaultProcessorsPerNode": 1, 
    "version": "0.0.1", 
    "defaultMaxRunTime": "01:00:00", 
    "available": True, 
    "inputs": [
        {
            "semantics": {
                "fileTypes": [
                    "text-0"
                ], 
                "minCardinality": 1, 
                "ontology": [
                    "http://sswapmeet.sswap.info/util/TextDocument"
                ], 
                "maxCardinality": 1
            }, 
            "id": "temp", 
            "value": {
                "default": "", 
                "required": True, 
                "enquote": False, 
                "visible": True, 
                "validator": "", 
                "order": 0
            }, 
            "details": {
                "repeatArgument": False, 
                "showArgument": False, 
                "description": "", 
                "label": "A text file."
            }
        }
    ], 
    "tags": [
        "containers", 
        "docker"
    ], 
    "outputs": [], 
    "longDescription": "", 
    "executionSystem": "my.execution.system", 
    "testPath": HOME_DIR,
    "deploymentPath": HOME_DIR, 
    "templatePath": "wc.sh", 
    "deploymentSystem": "my.test.system", 
    "name": "my.wc", 
    "checkpointable": True, 
    "executionType": "CLI", 
    "publiclyAvailable": "false", 
    "parallelism": "SERIAL", 
    "helpURI": "https://example.com"
}

Registering the app is similar to registering the systems:

In [26]:
agave.apps.add(body=app)

{u'_links': {u'executionSystem': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system'},
  u'metadata': {u'href': u'https://docker.example.com/meta/2.0/data/?q={"associationIds":"0001431722633030-242ac1111-0001-005"}'},
  u'owner': {u'href': u'https://docker.example.com/profiles/2.0/jdoe'},
  u'permissions': {u'href': u'https://docker.example.com/apps/2.0/my.wc-0.0.1/pems'},
  u'self': {u'href': u'https://docker.example.com/apps/2.0/my.wc-0.0.1'},
  u'storageSystem': {u'href': u'https://docker.example.com/systems/2.0/my.test.system'}},
 u'available': True,
 u'checkpointable': True,
 u'defaultMaxRunTime': u'01:00:00',
 u'defaultMemoryPerNode': 2,
 u'defaultNodeCount': 1,
 u'defaultProcessorsPerNode': 1,
 u'defaultQueue': u'debug',
 u'deploymentPath': u'/home/jstubbs',
 u'deploymentSystem': u'my.test.system',
 u'executionSystem': u'my.execution.system',
 u'executionType': u'CLI',
 u'helpURI': u'https://example.com',
 u'icon': None,
 u'id': u'my.wc-0.0.1',
 u'inputs': [{

We should see our new app in the list of apps:

In [33]:
[app.name for app in agave.apps.list()]

[u'pysdk-testsuitewc-wm_', u'pysdk-testsuitejfs_', u'my.wc', u'endofday-local']

We can access all the information of the app by its name together with its version:

In [34]:
agave.apps.get(appId='my.wc-0.0.1')

{u'_links': {u'executionSystem': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system'},
  u'metadata': {u'href': u'https://docker.example.com/meta/2.0/data/?q={"associationIds":"0001431722633030-242ac1111-0001-005"}'},
  u'owner': {u'href': u'https://docker.example.com/profiles/2.0/jdoe'},
  u'permissions': {u'href': u'https://docker.example.com/apps/2.0/my.wc-0.0.1/pems'},
  u'self': {u'href': u'https://docker.example.com/apps/2.0/my.wc-0.0.1'},
  u'storageSystem': {u'href': u'https://docker.example.com/systems/2.0/my.test.system'}},
 u'available': True,
 u'checkpointable': True,
 u'defaultMaxRunTime': u'01:00:00',
 u'defaultMemoryPerNode': 2,
 u'defaultNodeCount': 1,
 u'defaultProcessorsPerNode': 1,
 u'defaultQueue': u'debug',
 u'deploymentPath': u'/home/jstubbs',
 u'deploymentSystem': u'my.test.system',
 u'executionSystem': u'my.execution.system',
 u'executionType': u'CLI',
 u'helpURI': u'https://example.com',
 u'icon': None,
 u'id': u'my.wc-0.0.1',
 u'inputs': [{

### Jobs

Submit a job to exercise the app. We'll set as input the file `hello.txt` that we uploaded before:

In [48]:
job = {
  "name": "My Word Count",
  "appId": "my.wc-0.0.1",
  "inputs": {
    "temp": ["agave://my.test.system//{HOME_DIR}/hello.txt".format(HOME_DIR=HOME_DIR)]
  },
  "archive": True,
  "notifications": [
    {
      "url": "http://requestb.in/p9fm7ap9?job_id=${JOB_ID}&status=${JOB_STATUS}",
      "event": "*",
      "persistent": True
    }
  ]
}

my_job = agave.jobs.submit(body=job)
my_job

{u'_links': {u'app': {u'href': u'https://docker.example.com/apps/2.0/my.wc-0.0.1'},
  u'archiveData': {u'href': u'https://docker.example.com/jobs/2.0/0001431970348786-242ac1112-0001-007/outputs/listings'},
  u'archiveSystem': {u'href': u'https://docker.example.com/systems/2.0/test-storage-wm.tacc2'},
  u'executionSystem': {u'href': u'https://docker.example.com/systems/2.0/my.execution.system'},
  u'history': {u'href': u'https://docker.example.com/jobs/2.0/0001431970348786-242ac1112-0001-007/history'},
  u'metadata': {u'href': u'https://docker.example.com/meta/2.0/data/?q={"associationIds":"0001431970348786-242ac1112-0001-007"}'},
  u'notifications': {u'href': u'https://docker.example.com/notifications/2.0/?associatedUuid=0001431970348786-242ac1112-0001-007'},
  u'owner': {u'href': u'https://docker.example.com/profiles/2.0/jdoe'},
  u'permissions': {u'href': u'https://docker.example.com/jobs/2.0/0001431970348786-242ac1112-0001-007/pems'},
  u'self': {u'href': u'https://docker.example.co

We can retrieve the status of the job with:

In [49]:
agave.jobs.getStatus(jobId=my_job['id'])

{u'_links': {u'self': {u'href': u'https://docker.example.com/jobs/2.0/0001431970348786-242ac1112-0001-007'}},
 u'id': u'0001431970348786-242ac1112-0001-007',
 u'status': u'STAGING_INPUTS'}

Let's wait until the job is finished:

In [51]:
import time

while agave.jobs.getStatus(jobId=my_job['id'])['status'] != 'FINISHED':
    time.sleep(1)
    
print 'Done!'

Done!


Finally, we can retrieve the result that was left in the file `out.txt`:

In [52]:
agave.files.download(systemId='my.test.system', filePath=my_job['archivePath']+'/out.txt').text

u'2'

That's precisely the word count of "`Hello World!`"!!